In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras import Sequential
import scipy
import matplotlib.pyplot as plt
import csv

Using TensorFlow backend.


# read data

In [2]:
train_data = pd.read_csv("train.csv", encoding="big5")
print(train_data)

              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    16    17    18    19    20    21    22

In [3]:
test_data = pd.read_csv("test.csv",header=None)

r=range(test_data.shape[1]-2)
columns=['id','測向']
for i in r:
    columns.extend(str(i))
test_data.columns=columns
print(test_data)

          id          測向     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


# data preprocess

In [4]:
for i in range(10 ,len(train_data), 18):
    rainFall_row=train_data.iloc[i]
#     print(rainFall_row)
    for j in range(len(rainFall_row)):
        if rainFall_row[j]=='NR':
            rainFall_row[j]=0
        #print(rainFall_row[j])

In [5]:
train_y=train_data[train_data['測項'].isin(['PM2.5'])]
train_x=train_data[True^train_data['測項'].isin(['PM2.5'])]

train_x=train_x.iloc[:,3:].values
train_y=train_y.iloc[:,3:].values

train_x=np.array(train_x,dtype=np.float)
train_y=np.array(train_y,dtype=np.float)

print(train_x.shape)
print(train_y.shape)

(4080, 24)
(240, 24)


In [6]:
print(train_y.shape[0]*train_y.shape[1])

5760


In [7]:
train_x=np.reshape(train_x.transpose(),(train_x.shape[0]*train_x.shape[1]//17,17))
train_y=np.reshape(train_y, (train_y.shape[0]*train_y.shape[1],))

In [8]:
print("shape",train_x.shape)
print('======')
print(train_x)

shape (5760, 17)
[[ 14.     1.8    0.51 ...  35.     1.4    0.5 ]
 [ 16.     1.8    0.26 ... 108.     2.4    2.5 ]
 [ 18.     1.8    0.32 ... 111.     1.5    1.7 ]
 ...
 [ 14.     1.8    0.35 ... 123.     1.3    1.4 ]
 [ 15.     1.7    0.19 ...  22.     0.7    1.6 ]
 [ 13.     1.8    0.56 ... 105.     2.     2.  ]]


# train

In [9]:
model=keras.Sequential()
optimizer=keras.optimizers.RMSprop(0.001, rho=0.9)
# 加入normalize完全無效
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Dense(1,input_shape=(17,)))
# model.compile(loss="mse", optimizer='sgd')
model.compile(loss="mse", optimizer=optimizer)

In [10]:
model.fit(train_x, train_y,epochs=50)
model.save('weight.h5')







Epoch 1/50





5760/5760 [==============================] - 1s 125us/step - loss: 704.0524
Epoch 2/50
5760/5760 [==============================] - 0s 37us/step - loss: 626.7872
Epoch 3/50
5760/5760 [==============================] - 0s 36us/step - loss: 525.2320
Epoch 4/50
5760/5760 [==============================] - 0s 37us/step - loss: 417.5163
Epoch 5/50
5760/5760 [==============================] - 0s 37us/step - loss: 330.7184
Epoch 6/50
5760/5760 [==============================] - 0s 38us/step - loss: 285.5025
Epoch 7/50
5760/5760 [==============================] - 0s 38us/step - loss: 276.2937
Epoch 8/50
5760/5760 [==============================] - 0s 43us/step - loss: 275.6465
Epoch 9/50
5760/5760 [==============================] - 0s 39us/step - loss: 275.1447
Epoch 10/50
5760/5760 [==============================] - 0s 39us/step - loss: 274.2696
Epoch 11/50
5760/5760 [==============================] - 0s 36us/step - loss: 274.0699
Epoch 12/50
5760/5760 [=================

# test data

In [11]:
test_data = pd.read_csv("test.csv",header=None)
r=list(range(9))
columns=['id','測項']
for i in range(len(r)):
    columns.extend(str(r[i]))
test_data.columns=columns
print(test_data)

          id          測項     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


In [12]:
for i in range(10 ,len(test_data), 18):
    rainFall_row=test_data.iloc[i]
#     print(rainFall_row)
    for j in range(len(rainFall_row)):
        if rainFall_row[j]=='NR':
            rainFall_row[j]=0
#         print(rainFall_row[j])

In [13]:
print(test_data)

          id          測項     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


In [14]:
test_y=test_data[test_data['測項'].isin(['PM2.5'])]
test_x=test_data[True^test_data['測項'].isin(['PM2.5'])]

test_x=test_x.iloc[:,2:].values
test_y=test_y.iloc[:,2:].values
print(test_x)

test_x=np.array(test_x,dtype=np.float)
test_y=np.array(test_y,dtype=np.float)

print(test_x.shape)
print(test_y.shape)

[['21' '21' '20' ... '19' '18' '17']
 ['1.7' '1.7' '1.7' ... '1.7' '1.7' '1.8']
 ['0.39' '0.36' '0.36' ... '0.34' '0.31' '0.23']
 ...
 ['76' '99' '93' ... '98' '97' '65']
 ['2.2' '3.2' '2.5' ... '5.7' '4.9' '3.6']
 ['1.7' '2.8' '2.6' ... '4.9' '5.2' '3.6']]
(4080, 9)
(240, 9)


In [15]:
test_x=np.reshape(test_x.transpose(),(test_x.shape[0]*test_x.shape[1]//17,17))
test_y=np.reshape(test_y, (test_y.shape[0]*test_y.shape[1],))

In [16]:
print("X shape",test_x.shape)
print("Y shape",test_y.shape)

X shape (2160, 17)
Y shape (2160,)


In [17]:
result=model.predict(test_x)
print(result)

[[23.123352]
 [22.433868]
 [13.651312]
 ...
 [15.663336]
 [ 8.189766]
 [18.424501]]


In [18]:
f = open("my_output.csv","w")
w = csv.writer(f)
title = ['id','value']
w.writerow(title) 
for i in range(240):
    content = ['id_'+str(i),result[i][0]]
    w.writerow(content) 